In [7]:
# %pip install requests beautifulsoup4 nltk pandas

In [8]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\juneh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [9]:
import requests
from nltk.sentiment import SentimentIntensityAnalyzer

# 감성 분석기 초기화
sia = SentimentIntensityAnalyzer()

def fetch_news(company_name, start_date, end_date, client_id, client_secret):
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    
    news_articles = []
    params = {
        'query': company_name,
        'start': 1,
        'display': 100,  # 한 번에 최대 100개 기사 조회
        'sort': 'sim',  # 유사도 기준으로 정렬
        'begin': start_date,
        'end': end_date
    }
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        for item in data['items']:
            title = item['title']
            news_articles.append({'title': title})
    else:
        print("Error:", response.status_code, response.text)

    return news_articles

def analyze_sentiment(news_articles):
    sentiment_scores = []
    
    for article in news_articles:
        score = sia.polarity_scores(article['title'])
        sentiment_scores.append(score['compound'])  # compound score 저장
    
    return sentiment_scores

def calculate_sentiment_rate(sentiment_scores):
    positive_count = sum(1 for score in sentiment_scores if score > 0)
    negative_count = sum(1 for score in sentiment_scores if score < 0)
    
    total_count = len(sentiment_scores)
    if total_count == 0:
        return 0, 0  # 뉴스가 없는 경우 0 반환
    
    positive_rate = positive_count / total_count
    negative_rate = negative_count / total_count
    
    return positive_rate, negative_rate

def main(company_name):
    # API 클라이언트 ID와 시크릿 입력
    client_id = 'xy05yUbRMSx0DOOE9LUz'  # 여기에 클라이언트 ID 입력
    client_secret = 'LmS7qnabBm'  # 여기에 클라이언트 시크릿 입력
    
    # 날짜 범위 설정
    start_date = '20160101'  # 시작 날짜
    end_date = '20181231'    # 종료 날짜

    # 뉴스 크롤링
    news_articles = fetch_news(company_name, start_date, end_date, client_id, client_secret)

    # 감성 분석
    sentiment_scores = analyze_sentiment(news_articles)

    # 긍부정률 계산
    positive_rate, negative_rate = calculate_sentiment_rate(sentiment_scores)

    print(f"{company_name}의 긍정률: {positive_rate:.2f}, 부정률: {negative_rate:.2f}")

# 기업 이름을 직접 지정하여 호출
if __name__ == "__main__":
    company_name = '삼성전자'
    # company_name = input("기업 이름을 입력하세요: ")  # 예: 삼성전자
    main(company_name)


삼성전자의 긍정률: 0.00, 부정률: 0.00


In [1]:
import requests
import pandas as pd

def fetch_news(company_name, start_date, end_date, client_id, client_secret):
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    
    news_articles = []
    params = {
        'query': company_name,
        'start': 1,
        'display': 100,  # 한 번에 최대 100개 기사 조회
        'sort': 'sim',  # 유사도 기준으로 정렬
        'begin': start_date,
        'end': end_date
    }
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        for item in data['items']:
            title = item['title']
            news_articles.append({'title': title})
    else:
        print("Error:", response.status_code, response.text)

    return news_articles

def save_to_excel(news_articles, filename):
    df = pd.DataFrame(news_articles)
    df.to_excel(filename, index=False)

def main(company_name):
    # API 클라이언트 ID와 시크릿 입력
    client_id = 'xy05yUbRMSx0DOOE9LUz'  # 여기에 클라이언트 ID 입력
    client_secret = 'LmS7qnabBm'  # 여기에 클라이언트 시크릿 입력

    # 날짜 범위 설정
    start_date = '20160101'  # 시작 날짜
    end_date = '20181231'    # 종료 날짜

    # 뉴스 크롤링
    news_articles = fetch_news(company_name, start_date, end_date, client_id, client_secret)

    # 엑셀 파일로 저장
    save_to_excel(news_articles, f"{company_name}_news.xlsx")
    print(f"{company_name}에 대한 뉴스 기사가 '{company_name}_news.xlsx'로 저장되었습니다.")

# 기업 이름을 직접 지정하여 호출
if __name__ == "__main__":
    company_name = '삼성전자'
    main(company_name)


삼성전자에 대한 뉴스 기사가 '삼성전자_news.xlsx'로 저장되었습니다.


In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def fetch_news(company_name, start_date, end_date, client_id, client_secret):
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    
    news_articles = []
    params = {
        'query': company_name,
        'start': 1,
        'display': 100,  # 한 번에 최대 100개 기사 조회
        'sort': 'sim',  # 유사도 기준으로 정렬
        'begin': start_date,
        'end': end_date
    }
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        for item in data['items']:
            title = item['title']
            link = item['link']  # 기사 링크 추가
            pub_date = item['pubDate']  # 기사 날짜 추가
            
            # 기사 본문 크롤링
            article_content = fetch_article_content(link)
            
            # 날짜에서 연도, 월, 일 추출
            date_object = datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S %z")
            year = date_object.year
            month = date_object.month
            day = date_object.day
            
            news_articles.append({
                'year': year,
                'month': month,
                'day': day,
                'title': title,
                'content': article_content
            })
    else:
        print("Error:", response.status_code, response.text)

    return news_articles

def fetch_article_content(link):
    try:
        response = requests.get(link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # 본문 내용 추출
            content = soup.select_one('.news_end')  # 네이버 뉴스 본문 선택자
            return content.get_text(strip=True) if content else "본문을 찾을 수 없습니다."
        else:
            return "기사 본문을 불러오는 데 실패했습니다."
    except Exception as e:
        return f"오류 발생: {e}"

def save_to_excel(news_articles, filename):
    df = pd.DataFrame(news_articles)
    df.to_excel(filename, index=False)

def main(company_name):
    # API 클라이언트 ID와 시크릿 입력
    client_id = 'xy05yUbRMSx0DOOE9LUz'  # 여기에 클라이언트 ID 입력
    client_secret = 'LmS7qnabBm'  # 여기에 클라이언트 시크릿 입력

    # 날짜 범위 설정
    start_date = '20160101'  # 시작 날짜
    end_date = '20181231'    # 종료 날짜

    # 뉴스 크롤링
    news_articles = fetch_news(company_name, start_date, end_date, client_id, client_secret)

    # 엑셀 파일로 저장
    save_to_excel(news_articles, f"{company_name}_news.xlsx")
    print(f"{company_name}에 대한 뉴스 기사가 '{company_name}_news.xlsx'로 저장되었습니다.")

# 기업 이름을 직접 지정하여 호출
if __name__ == "__main__":
    company_name = '삼성전자'
    main(company_name)


삼성전자에 대한 뉴스 기사가 '삼성전자_news.xlsx'로 저장되었습니다.
